In [14]:
###install.packages("tidygenomics")
#install.packages("dplyr")
#install.packages("fuzzyjoin")
#install.packages("readxl")
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install("IRanges")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.0 (2021-05-18)

Installing package(s) 'BiocVersion', 'IRanges'

also installing the dependencies ‘BiocGenerics’, ‘S4Vectors’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'fansi'



In [1]:
#library(tidygenomics)
library(dplyr)
library(fuzzyjoin)
library(readxl)
library(IRanges)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
 

In [2]:
## Read in tables with identified germinal Mu insertions; 
all_ins <- read.csv("../../results/insertions_table_final/all_identified_insertions.csv", header=TRUE)



## Read Annotation File
#setwd("../FGS/")
#MY_gff3 <- list.files(getwd(), pattern="\\.gff3$")
annotation <- read.delim("../../resources/final_annotation_table", header=FALSE, comment.char="#")
# add column names
colnames(annotation) <- c("GeneID", "Chr", "Start", "End")

# reduce genes to ONE entry which is the longest
# multiple entries per gene likely, since after gffread, transcripts of different lengths are included (share name)
# we find the min and max positions (start & end coordinate with greatest distance) per gene and keep this record  
annotation <- annotation %>%
  group_by(GeneID, Chr) %>%
  summarise(Start = min(Start), End = max(End))

# compute gene_length
annotation <- annotation %>% 
  mutate(Gene_length = End - Start + 1)


# relocate Gene column before gene_length
#annotation <- annotation %>%
#  relocate(Gene, .before = gene_length)

`summarise()` has grouped output by 'GeneID'. You can override using the `.groups` argument.



In [4]:
head(annotation)
extension = 100
annotation <- annotation %>%
  mutate(Start = Start -extension, End = End + extension)
head(annotation)

GeneID,Chr,Start,End,Gene_length
<chr>,<chr>,<int>,<int>,<dbl>
ENSRNA049437471,1,2815814,2815887,74
ENSRNA049437473,1,2998702,2998773,72
ENSRNA049437518,1,5888817,5888890,74
ENSRNA049437607,1,10109832,10109903,72
ENSRNA049437614,1,10573120,10573190,71
ENSRNA049437658,1,18379795,18379867,73


GeneID,Chr,Start,End,Gene_length
<chr>,<chr>,<dbl>,<dbl>,<dbl>
ENSRNA049437471,1,2815714,2815987,74
ENSRNA049437473,1,2998602,2998873,72
ENSRNA049437518,1,5888717,5888990,74
ENSRNA049437607,1,10109732,10110003,72
ENSRNA049437614,1,10573020,10573290,71
ENSRNA049437658,1,18379695,18379967,73


In [18]:
head(all_ins)
head(annotation)
nrow(annotation)

,Chr,Start,End,Sample,StartReads,EndReads
,<chr>,<int>,<int>,<chr>,<int>,<int>
1,1,262679,262687,Col-01,3,13
2,1,1231462,1231470,Col-01,39,3
3,1,2113392,2113400,Col-01,37,8
4,1,2113407,2113415,Col-01,18,2
5,1,2170572,2170580,Col-01,41,4
6,1,2232236,2232244,Col-01,2,2


GeneID,Chr,Start,End,Gene_length
<chr>,<chr>,<int>,<int>,<dbl>
ENSRNA049437471,1,2815814,2815887,74
ENSRNA049437473,1,2998702,2998773,72
ENSRNA049437518,1,5888817,5888890,74
ENSRNA049437607,1,10109832,10109903,72
ENSRNA049437614,1,10573120,10573190,71
ENSRNA049437658,1,18379795,18379867,73


[1] 46430

In [36]:
# join MuGerminal table with annotation
all_ins_annotated <- fuzzyjoin::genome_inner_join(all_ins, 
                                annotation, 
                                by=c("Chr", "Start", "End") 
                                )
head(all_ins_annotated)

,Chr.x,Start.x,End.x,Sample,StartReads,EndReads,GeneID,Chr.y,Start.y,End.y,Gene_length
,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<dbl>
1,1,1231462,1231470,Col-01,39,3,Zm00001d027276,1,1226063,1236227,10165
2,1,2113392,2113400,Col-01,37,8,Zm00001d027296,1,2109189,2113478,4290
3,1,2113407,2113415,Col-01,18,2,Zm00001d027296,1,2109189,2113478,4290
4,1,2170572,2170580,Col-01,41,4,Zm00001d022653,1,2170280,2170983,704
5,1,2809859,2809867,Col-01,43,13,Zm00001d027314,1,2809751,2811917,2167
6,1,2869732,2869740,Col-01,37,8,Zm00001d027319,1,2869707,2878262,8556


In [37]:
# rename, relocate and drop the columns to create a useful table
all_ins_annotated <- all_ins_annotated %>%
  dplyr::rename(
    Chr = "Chr.y",
    Start = "Start.y",
    End = "End.y",
    InsertionStart = "Start.x",
    InsertionEnd = "End.x"
  ) %>%
  relocate(
    Sample, InsertionStart, InsertionEnd, StartReads, EndReads, .before = Gene_length
  ) %>%
  select(
    -Chr.x
  )

In [38]:
# order the table as to have intersecting samples next to one another
all_ins_annotated <- all_ins_annotated %>%
  arrange(., Chr, Start, InsertionStart)

In [39]:
#setwd("MuSeq_table_final/")
write.csv(all_ins_annotated, 
          "../../results/insertions_table_final/all_identified_insertions_annotated.csv", 
          row.names=F)